In [1]:
import sys, os
import pandas as pd
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

import time
from tqdm import tqdm

import warnings
warnings.filterwarnings(
    "ignore",
    message="X does not have valid feature names, but LGBMClassifier was fitted with feature names",
    category=UserWarning,
    module="sklearn",
)

from sklearn import metrics
from scipy import stats
from flaml import AutoML # https://microsoft.github.io/FLAML/docs/Use-Cases/Task-Oriented-AutoML/

import torch
torch.manual_seed(0)

import matplotlib.pyplot as plt
import seaborn as sns
from itertools import compress

/work3/s232958/envs/esm_cuda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'Could not load this library: /work3/s232958/envs/esm_cuda/lib/python3.10/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/work3/s232958/envs/esm_cuda/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
Df_train_small = pd.read_csv("/work3/s232958/data/PPint_DB/PPint_train.csv",index_col=0).reset_index(drop=True)
Df_train_small = Df_train_small[~Df_train_small.target_binder_id.str.startswith("6BJP")]
Df_train_small["interface_id"] = [row.ID1.split("_")[0]+"_"+row.ID1.split("_")[1] for __, row in Df_train_small.iterrows()]

Df_test_small = pd.read_csv("/work3/s232958/data/PPint_DB/PPint_test.csv",index_col=0).reset_index(drop=True)
Df_test_small["interface_id"] = [row.ID1.split("_")[0]+"_"+row.ID1.split("_")[1] for __, row in Df_test_small.iterrows()] 

Df_train = pd.read_csv("/work3/s232958/data/PPint_DB/PPint_train_w_pbd_lens.csv",index_col=0).reset_index(drop=True)
Df_train = Df_train.merge(Df_train_small[["dimer", "interface_id"]], on = "interface_id", how="inner")

Df_test = pd.read_csv("/work3/s232958/data/PPint_DB/PPint_test_w_pbd_lens.csv",index_col=0).reset_index(drop=True)
Df_test = Df_test.merge(Df_test_small[["dimer", "interface_id"]], on = "interface_id", how="inner")
Df_test["binder_id"] = [row.ID1.split("_")[0]+"_"+row.ID1.split("_")[2] for __, row in Df_test.iterrows()]
Df_test["target_id"] = [row.ID2.split("_")[0]+"_"+row.ID2.split("_")[2] for __, row in Df_test.iterrows()] 

Df_train = Df_train[~Df_train.PDB.str.startswith("6BJP")]
Df_train["binder_id"] = [row.ID1.split("_")[0]+"_"+row.ID1.split("_")[2] for __, row in Df_train.iterrows()]
Df_train["target_id"] = [row.ID2.split("_")[0]+"_"+row.ID2.split("_")[2] for __, row in Df_train.iterrows()]
Df_test

,interface_id,PDB,ID1,ID2,seq_target,seq_target_len,seq_pdb_target,pdb_target_len,target_chain,seq_binder,seq_binder_len,seq_pdb_binder,pdb_binder_len,binder_chain,pdb_path,dimer,binder_id,target_id
0,1NNW_0,1NNW,1NNW_0_A,1NNW_0_B,VYVAVLANIAGNLPALTAALSRIEEMREEGYEIEKYYILGNIVGLF...,251,VYVAVLANIAGNLPALTAALSRIEEMREEGYEIEKYYILGNIVGLF...,251,A,VYVAVLANIAGNLPALTAALSRIEEMREEGYEIEKYYILGNIVGLF...,251,MVYVAVLANIAGNLPALTAALSRIEEMREEGYEIEKYYILGNIVGL...,252,B,1nnw.pdb.gz,True,1NNW_A,1NNW_B
1,3UCN_0,3UCN,3UCN_0_A,3UCN_0_B,TADLSPLLEANRKWADECAAKDSTYFSKVAGSQAPEYLYIGCADSR...,222,TADLSPLLEANRKWADECAAKDSTYFSKVAGSQAPEYLYIGCADSR...,222,A,TADLSPLLEANRKWADECAAKDSTYFSKVAGSQAPEYLYIGCADSR...,222,TADLSPLLEANRKWADECAAKDSTYFSKVAGSQAPEYLYIGCADSR...,222,B,3ucn.pdb.gz,True,3UCN_A,3UCN_B
2,1POV_1,1POV,1POV_1_1,1POV_1_3,QHRSRSESSIESFFARGACVTIMTVDNPASTTNKDKLFAVWKITYK...,235,QHRSRSESSIESFFARGACVTIMTVDNPASTTNKDKLFAVWKITYK...,235,1,GLPVMNTPGSNQYLTADNFQSPCALPEFDVTPPIDIPGEVKNMMEL...,238,GLPVMNTPGSNQYLTADNFQSPCALPEFDVTPPIDIPGEVKNMMEL...,238,3,1pov.pdb.gz,False,1POV_1,1POV_3
3,3R6Y_2,3R6Y,3R6Y_2_C,3R6Y_2_D,VRIEKDFLGEKEIPKDAYYGVQTIRATENFPITGYRIHPELIKSLG...,383,VRIEKDFLGEKEIPKDAYYGVQTIRATENFPITGYRIHPELIKSLG...,383,C,VRIEKDFLGEKEIPKDAYYGVQTIRATENFPITGYRIHPELIKSLG...,390,VRIEKDFLGEKEIPKDAYYGVQTIRATENFPITGYRIHPELIKSLG...,390,D,3r6y.pdb.gz,True,3R6Y_C,3R6Y_D
4,5YHI_0,5YHI,5YHI_0_A,5YHI_0_B,PMRYPVDVYTGKIQVDGELMLTELGLEGDGPDRALCHYPREHYLYW...,202,PMRYPVDVYTGKIQVDGELMLTELGLEGDGPDRALCHYPREHYLYW...,202,A,PMRYPVDVYTGKIQVDGELMLTELGLEGDGPDRALCHYPREHYLYW...,201,PMRYPVDVYTGKIQVDGELMLTELGLEGDGPDRALCHYPREHYLYW...,201,B,5yhi.pdb.gz,True,5YHI_A,5YHI_B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,3GXE_0,3GXE,3GXE_0_B,3GXE_0_F,DQCIVDDITYNVQDTFHKKHEEGHMLNCTCFGQGRGRWKCDPVDQC...,89,DQCIVDDITYNVQDTFHKKHEEGHMLNCTCFGQGRGRWKCDPVDQC...,89,B,GLGMKGHRGF,10,GLPGMKGHRGF,11,F,3gxe.pdb.gz,False,3GXE_B,3GXE_F
490,6LY5_21,6LY5,6LY5_21_d,6LY5_21_l,PSPIFGGSTGGWLRKAQVEEKYVITWDSPKEQIFEMPTGGAAIMRE...,132,PSPIFGGSTGGWLRKAQVEEKYVITWDSPKEQIFEMPTGGAAIMRE...,132,d,ANFIKPYNDDPFVGHLATPITSSAVTRSLLKNLPAYRFGLTPLLRG...,144,ANFIKPYNDDPFVGHLATPITSSAVTRSLLKNLPAYRFGLTPLLRG...,144,l,6ly5.pdb.gz,False,6LY5_d,6LY5_l
491,5MLK_0,5MLK,5MLK_0_A,5MLK_0_B,ARISKVLVANRGEIAVRVIRAARDAGLPSVAVYAEPDAESPHVRLA...,451,ARISKVLVANRGEIAVRVIRAARDAGLPSVAVYAEPDAESPHVRLA...,451,A,ARISKVLVANRGEIAVRVIRAARDAGLPSVAVYAEPDAESPHVRLA...,384,ARISKVLVANRGEIAVRVIRAARDAGLPSVAVYAEPDAESPHVRLA...,384,B,5mlk.pdb.gz,True,5MLK_A,5MLK_B
492,8BS4_0,8BS4,8BS4_0_A,8BS4_0_B,HPVLEKLKAAHSYNPKEFEWNLKSGRVFIIKSYSEDDIHRSIKYSI...,195,HPVLEKLKAAHSYNPKEFEWNLKSGRVFIIKSYSEDDIHRSIKYSI...,195,A,GHPVLEKLKAAHSYNPKEFEWNLKSGRVFIIKSYSEDDIHRSIKYS...,193,GHPVLEKLKAAHSYNPKEFEWNLKSGRVFIIKSYSEDDIHRSIKYS...,193,B,8bs4.pdb.gz,True,8BS4_A,8BS4_B


In [3]:
encodings_path = "/work3/s232958/data/PPint_DB/embeddings_esmC"

binders_train = list(Df_train.binder_id)
binders_train_embedings = [np.load(os.path.join(encodings_path, f"{file}.npy"))[0] for file in binders_train]

targets_train = list(Df_train.target_id)
targets_train_embedings = [np.load(os.path.join(encodings_path, f"{file}.npy"))[0] for file in targets_train]

binders_test = list(Df_test.binder_id)
binders_test_embedings = [np.load(os.path.join(encodings_path, f"{file}.npy"))[0] for file in binders_test]

targets_test = list(Df_test.target_id)
targets_test_embedings = [np.load(os.path.join(encodings_path, f"{file}.npy"))[0] for file in targets_test]

In [11]:
### 1. Pooling function (Mean Pooling over the sequence length)
def pool_embeddings(embeddings_list):
    # Assumes input is a list of arrays with shape (L, 1152)
    # The [0] index in your loading code suggests you might already be taking a specific token,
    # but mean pooling is safer if L > 1.
    return np.array([np.mean(e, axis=0) if e.ndim > 1 else e for e in embeddings_list])

X_binder_train = pool_embeddings(binders_train_embedings)   # Shape: (N, 1152)
X_target_train = pool_embeddings(targets_train_embedings)   # Shape: (N, 1152)

X_binder_test = pool_embeddings(binders_test_embedings)     # Shape: (M, 1152)
X_target_test = pool_embeddings(targets_test_embedings)     # Shape: (M, 1152)

def prepare_balanced_no_duplicates(X_binder, X_target):
    # Calculate the midpoint to split the data
    n_total = len(X_binder)
    mid = n_total // 2
    
    # --- 1. Create Positive Half ---
    # We take the first half of the binders and targets as they are
    X_pos = np.concatenate([X_binder[:mid], X_target[:mid]], axis=1)
    y_pos = np.ones(mid)
    
    # --- 2. Create Negative Half ---
    # We take the second half of the binders and targets
    X_binder_neg_half = X_binder[mid:]
    X_target_neg_half = X_target[mid:]
    
    # Shift (roll) only the second half to create non-interacting pairs
    X_binder_shifted = np.roll(X_binder_neg_half, shift=1, axis=0)
    
    X_neg = np.concatenate([X_binder_shifted, X_target_neg_half], axis=1)
    y_neg = np.zeros(len(X_neg))
    
    # --- 3. Combine ---
    X_final = np.vstack([X_pos, X_neg])
    y_final = np.concatenate([y_pos, y_neg])
    
    return X_final, y_final

# Assuming X_binder_train, X_target_train etc. are already mean-pooled from your previous step:

# Process Train Set
X_train, y_train = prepare_balanced_no_duplicates(X_binder_train, X_target_train)
X_train, y_train = shuffle(X_train, y_train, random_state=0)

# Process Test Set
X_test, y_test = prepare_balanced_no_duplicates(X_binder_test, X_target_test)
X_test, y_test = shuffle(X_test, y_test, random_state=0)

print(f"Train size: {len(X_train)} ({int(sum(y_train))} Pos, {int(len(y_train)-sum(y_train))} Neg)")
print(f"Test size: {len(X_test)} ({int(sum(y_test))} Pos, {int(len(y_test)-sum(y_test))} Neg)")

Train size: 1977 (988 Pos, 989 Neg)
Test size: 494 (247 Pos, 247 Neg)


In [13]:
automl = AutoML()
settings = {
    "time_budget": 600,
    "metric": 'ap',               
    "task": 'classification',
    "eval_method": 'cv',          
    "n_splits": 5,                
    "split_type": 'stratified',   # Ensure classes are balanced in CV folds
    "estimator_list": ['lgbm', 'rf', 'extra_tree', 'xgboost'], # Focus on tree-models
    "early_stop": True,
    "verbose": 2,                 # Increased verbose to see what's happening
}

print("Starting AutoML Search...")
automl.fit(X_train=X_train, y_train=y_train, **settings)

# Evaluate on Test Set
y_pred = automl.predict(X_test)
y_proba = automl.predict_proba(X_test)[:, 1]

print(f"\nBest Estimator: {automl.best_estimator}")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Test AUROC: {roc_auc_score(y_test, y_proba):.4f}")
print(f"Test AP: {average_precision_score(y_test, y_proba):.4f}")

Starting AutoML Search...

Best Estimator: lgbm
Test Accuracy: 0.8927
Test AUROC: 0.9411
Test AP: 0.9532


In [15]:
automl = AutoML()
settings = {
    "time_budget": 600,
    "metric": 'ap',               
    "task": 'classification',
    "eval_method": 'cv',          
    "n_splits": 5,                
    "split_type": 'stratified',   # Ensure classes are balanced in CV folds
    # "estimator_list": ['lgbm', 'rf', 'extra_tree', 'xgboost'], # Focus on tree-models
    # "early_stop": True,
    "verbose": 2,                 # Increased verbose to see what's happening
}

print("Starting AutoML Search...")
automl.fit(X_train=X_train, y_train=y_train, **settings)

# Evaluate on Test Set
y_pred = automl.predict(X_test)
y_proba = automl.predict_proba(X_test)[:, 1]

print(f"\nBest Estimator: {automl.best_estimator}")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Test AUROC: {roc_auc_score(y_test, y_proba):.4f}")
print(f"Test AP: {average_precision_score(y_test, y_proba):.4f}")

Starting AutoML Search...

Best Estimator: lgbm
Test Accuracy: 0.8927
Test AUROC: 0.9411
Test AP: 0.9532
